# CSV to SQLite

This notebook shows how to `crawl` a directory of `*.csv` files and import them into SQLite.

In this example, we store `*.csv` files in a folder relative to the notebook `./data`. We then create a SQLite database from these CSV files.

To customize this notebook, change the values of the 2 variables below.

- `dir_to_csv`
- `sqlite_db`

## Create SQLite database

In [1]:
import pathlib
import pandas as pd
import sqlite3

def get_create_sql(file_path, df):
    def get_type(t):
        if t.startswith('int') or t.startswith('bool'):
            return 'INTEGER'
        if t.startswith('float'):
            return 'REAL'
        return 'TEXT'
    
    fields = [(n, get_type(str(t))) for n, t in zip(df.dtypes.index, df.dtypes.values)]
    fields = [f'{n} {t}' for n, t in fields]
    fields = ', '.join(fields)
    
    table = file_path.stem
    
    sql = f'CREATE TABLE {table} ({fields})'
    return sql
    
def get_data(file_path):
    df = pd.read_csv(file_path, low_memory=False)
    return df
        
def get_csv_files(dir_path):
    csv_files = list(pathlib.Path(dir_path).glob('*.csv'))
    return csv_files

def create_db(input_dir, output_db):
    pathlib.Path(output_db).touch()
    
    files = get_csv_files(input_dir)
    with sqlite3.connect(output_db) as conn:
        cur = conn.cursor()
        
        for file_path in files:
            print(f'processing {file_path}')
            
            df = get_data(file_path)
            
            sql = get_create_sql(file_path, df)
            cur.execute(sql)
            
            df.to_sql(file_path.stem, conn, if_exists='replace', index=False)

dir_to_csv = './data'
sqlite_db = './mydb.db'

create_db(dir_to_csv, sqlite_db)

processing data/grade.csv
processing data/person.csv


## Verify that it works with SQLite drive

In [2]:
with sqlite3.connect('./mydb.db') as conn:
    sql = '''
    SELECT *
    FROM person p
        JOIN grade g ON p.id = g.person_id
    '''
    
    cur = conn.cursor()
    cur.execute(sql)
    
    items = cur.fetchall()
    for i in items:
        print(i)

(1, 'john', 'smith', 32, 155.5, 1, 1, 1, 99.9)
(1, 'john', 'smith', 32, 155.5, 1, 2, 1, 88.8)
(1, 'john', 'smith', 32, 155.5, 1, 3, 1, 92.2)
(2, 'jane', 'clinton', 28, 133.3, 0, 4, 2, 88.9)
(2, 'jane', 'clinton', 28, 133.3, 0, 5, 2, 92.2)
(2, 'jane', 'clinton', 28, 133.3, 0, 6, 2, 84.4)


## Verify that it works with Pandas

In [3]:
with sqlite3.connect('./mydb.db') as conn:
    sql = '''
    SELECT *
    FROM person p
        JOIN grade g ON p.id = g.person_id
    '''
    
    df = pd.read_sql(sql, conn)
    
df

,id,first_name,last_name,age,weight,is_male,id,person_id,score
0,1,john,smith,32,155.5,1,1,1,99.9
1,1,john,smith,32,155.5,1,2,1,88.8
2,1,john,smith,32,155.5,1,3,1,92.2
3,2,jane,clinton,28,133.3,0,4,2,88.9
4,2,jane,clinton,28,133.3,0,5,2,92.2
5,2,jane,clinton,28,133.3,0,6,2,84.4


In [4]:
with sqlite3.connect('./mydb.db') as conn:
    sql = '''
    SELECT p.id, AVG(g.score) as score
    FROM person p
        JOIN grade g ON p.id = g.person_id
    GROUP BY p.id
    '''
    
    df = pd.read_sql(sql, conn)
    
df

,id,score
0,1,93.633333
1,2,88.500000


## Other applications

You may use other applications to open the SQLite database.

- [SQLite Studio](https://sqlitestudio.pl/)
  - [Install SQLite Studio using brew on Mac](https://formulae.brew.sh/cask/sqlitestudio)
- [Online SQLite Viewer](https://inloop.github.io/sqlite-viewer/)